In [14]:
import pandas as pd

In [15]:
df = pd.read_parquet("../data/processed/individual_2020_li.parquet")

In [16]:
df.ZIP_CODE.apply(lambda x: len(x)).value_counts()

ZIP_CODE
9    638797
5    320611
Name: count, dtype: int64

In [17]:
import geopandas as gpd
zipcodes_gdf = gpd.read_file("../references/long_island_zipcodes.geojson")

In [18]:
df["postalcode"] = df.ZIP_CODE.str[:5]

In [29]:
geocoded_df = df.merge(zipcodes_gdf[["postalcode", "geometry"]])
geocoded_gdf = gpd.GeoDataFrame(geocoded_df, geometry = geocoded_df["geometry"], crs=4326)

In [45]:
geocoded_gdf.iloc[0]

CMTE_ID                                               C00401224
AMNDT_IND                                                     N
RPT_TP                                                       MY
TRANSACTION_PGI                                            None
IMAGE_NUM                                    201907299154766847
TRANSACTION_TP                                              24T
ENTITY_TP                                                   IND
NAME                                            JOINNIDES, ANNE
CITY                                                      ISLIP
STATE                                                        NY
ZIP_CODE                                                  11751
EMPLOYER                                           NOT EMPLOYED
OCCUPATION                                         NOT EMPLOYED
TRANSACTION_DT                                         02282019
TRANSACTION_AMT                                              25
OTHER_ID                                

In [44]:
geocoded_gdf["TransactionAmt"] = geocoded_gdf["TRANSACTION_AMT"].astype(float)

8&Be careful, this one can take a while! about a minute9&1

In [32]:
import geopandas as gpd
towns_gdf = gpd.read_file("../references/shapefiles/Cities_Towns.shp").to_crs(4326)

In [37]:
geocoded_gdf

,CMTE_ID,AMNDT_IND,RPT_TP,TRANSACTION_PGI,IMAGE_NUM,TRANSACTION_TP,ENTITY_TP,NAME,CITY,STATE,...,TRANSACTION_DT,TRANSACTION_AMT,OTHER_ID,TRAN_ID,FILE_NUM,MEMO_CD,MEMO_TEXT,SUB_ID\n,postalcode,geometry
0,C00401224,N,MY,None,201907299154766847,24T,IND,"JOINNIDES, ANNE",ISLIP,NY,...,02282019,25,C00000935,SA11AI_148873367,1344765,None,EARMARKED FOR DCCC (C00000935),4082820191123930578,11751,POINT (-73.21424 40.73609)
1,C00401224,N,MY,None,201907299154766847,24T,IND,"JOINNIDES, ANNE",ISLIP,NY,...,03062019,15,C00000935,SA11AI_149487838,1344765,None,EARMARKED FOR DCCC (C00000935),4082820191123930579,11751,POINT (-73.21424 40.73609)
2,C00401224,N,MY,None,201907299154766848,24T,IND,"JOINNIDES, ANNE",ISLIP,NY,...,03292019,25,C00000935,SA11AI_152069691,1344765,None,EARMARKED FOR DCCC (C00000935),4082820191123930581,11751,POINT (-73.21424 40.73609)
3,C00401224,N,MY,None,201907299154766848,24T,IND,"JOINNIDES, ANNE",ISLIP,NY,...,04062019,15,C00000935,SA11AI_153371061,1344765,None,EARMARKED FOR DCCC (C00000935),4082820191123930582,11751,POINT (-73.21424 40.73609)
4,C00401224,N,MY,None,201907299154766845,24T,IND,"JOINNIDES, ANNE",ISLIP,NY,...,01062019,15,C00000935,SA11AI_144624917,1344765,None,EARMARKED FOR DCCC (C00000935),4082820191123930573,11751,POINT (-73.21424 40.73609)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
959403,C00401224,N,30G,None,202012049357342196,24T,IND,"PALMER, MARK",UPTON,NY,...,10292020,25,C00633248,SA11AI_340221550,1481637,None,EARMARKED FOR 314 ACTION FUND (C00633248),4010420212000914015,11973,POINT (-72.88065 40.86329)
959404,C00010603,A,YE,G2020,202101319424605549,15E,IND,"TUOZZOLO, JOSEPH",UPTON,NY,...,12132020,1000,C00401224,38479029,1497317,None,* EARMARKED CONTRIBUTION: SEE BELOW,4020320212017767423,11973,POINT (-72.88065 40.86329)
959405,C00401224,N,YE,None,202101319420539586,24T,IND,"PALMER, MARK",UPTON,NY,...,12112020,25,C00633248,SA11AI_360564543,1496802,None,EARMARKED FOR 314 ACTION FUND (C00633248),4031120211249865804,11973,POINT (-72.88065 40.86329)
959406,C00633248,A,YE,P,202104129443235174,15E,IND,"PALMER, MARK",UPTON,NY,...,12112020,25,None,VNW44JXGSD3,1509550,None,* EARMARKED CONTRIBUTION: SEE BELOW,4041520211277566112,11973,POINT (-72.88065 40.86329)


In [46]:
town_data = gpd.sjoin(geocoded_gdf, towns_gdf, predicate="within")

In [48]:
town_spending = town_data.groupby("NAME_right")["TransactionAmt"].sum().reset_index().rename(columns={"NAME_right": "NAME"})
town_spending

,NAME,TransactionAmt
0,Babylon,2326575.0
1,Brookhaven,15012779.0
2,East Hampton,5758347.0
3,Glen Cove,806447.0
4,Hempstead,17534222.0
5,Huntington,9105101.0
6,Islip,5147893.0
7,Long Beach,1016479.0
8,North Hempstead,16045870.0
9,Oyster Bay,16843158.0


In [54]:
from keplergl import KeplerGl
town_spending_map = KeplerGl(height=800)
town_spending_map.add_data(towns_gdf.merge(town_spending)[["NAME", "TransactionAmt", "geometry"]], name="Town")
town_spending_map

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'Town': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14], 'columns': ['NAME', 'Tran…

In [5]:
from keplergl import KeplerGl
county_map = KeplerGl(height=800)
county_map.add_data(towns_gdf[towns_gdf.COUNTY=='Nassau'])
county_map

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


/Users/davidgallagher/Code/src/github.com/oneextrafact/pydata-federal-contributions/.venv/lib/python3.10/site-packages/jupyter_client/session.py:719: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


KeplerGl(data={'unnamed': {'index': [343, 401, 508, 629, 674], 'columns': ['NAME', 'MUNI_TYPE', 'MUNITYCODE', …

In [6]:
villages_gdf = gpd.read_file("../references/shapefiles/Villages.shp")
villages_gdf

,NAME,TOWN,COUNTY,GNIS_ID,FIPS_CODE,SWIS,POP1990,POP2000,POP2010,POP2020,DOS_LL,DOSLL_DATE,MAP_SYMBOL,CALC_SQ_MI,DATEMOD,Shape_Leng,Shape_Area,geometry
0,Adams,Adams,Jefferson,2391499,360450021000199,222001,1753,1624,1775,1633,1-2005,2005-04-25,1,1.449596,2018-11-05,10475.019960,3.754438e+06,"POLYGON ((418500.196 4852516.977, 418496.081 4..."
1,Addison,Addison,Steuben,2391500,361010028700276,462001,1842,1797,1763,1561,NaN,NaN,1,1.930132,2018-02-28,12848.275937,4.999019e+06,"POLYGON ((316042.520 4665479.385, 316052.468 4..."
2,Afton,Afton,Chenango,2391501,360170035300342,082001,838,836,822,794,NaN,NaN,1,1.629228,2018-10-02,8343.865424,4.219682e+06,"POLYGON ((457500.805 4676527.052, 457530.173 4..."
3,Airmont,Ramapo,Rockland,2391502,360876051000408,392621,1770,7799,8628,10166,NaN,NaN,3,4.563626,2019-10-04,15615.663926,1.181974e+07,"POLYGON ((578157.706 4550468.971, 578180.609 4..."
4,Akron,Newstead,Erie,2391503,360295071600441,145601,2906,3085,2868,2888,NaN,NaN,1,1.927304,2021-07-15,18609.811738,4.991694e+06,"POLYGON ((214625.772 4770075.179, 214615.018 4..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
528,Woodsburgh,Hempstead,Nassau,2391226,360593400082986,282037,1190,831,778,897,NaN,NaN,3,0.418232,2021-08-17,5764.042194,1.083215e+06,"POLYGON ((609360.730 4498013.340, 609389.690 4..."
529,Wurtsboro,Mamakating,Sullivan,2391227,361054480983272,484003,1048,1234,1246,1124,NaN,NaN,1,1.249751,2018-11-21,7245.240267,3.236841e+06,"POLYGON ((544154.400 4603004.506, 543317.492 4..."
530,Wyoming,Middlebury,Wyoming,2391228,361214687783371,563801,478,513,434,377,NaN,NaN,1,0.670962,2018-09-20,5338.545696,1.737785e+06,"POLYGON ((248331.232 4745306.070, 248023.215 4..."
531,Yorkville,Whitestown,Oneida,2391229,360658175484099,307005,2972,2675,2689,2657,NaN,NaN,2,0.678641,2020-07-15,6867.765567,1.757671e+06,"POLYGON ((477128.834 4773116.032, 477134.314 4..."


In [10]:
village_map = KeplerGl(height=800)
village_map.add_data(villages_gdf[villages_gdf.COUNTY=='Nassau'][["NAME", "TOWN", "geometry"]])
village_map

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'unnamed': {'index': [25, 36, 37, 39, 57, 85, 88, 115, 139, 142, 144, 159, 164, 166, 176, 181, …

In [28]:
df["EMPLOYER"].value_counts().head(20)

EMPLOYER
RETIRED                             223831
NOT EMPLOYED                        200383
SELF-EMPLOYED                        37976
SELF                                 34942
NONE                                 34203
N/A                                  27777
SELF EMPLOYED                        13506
INFORMATION REQUESTED                 4305
FEDERAL AVIATION ADMINISTRATION       3624
STONY BROOK UNIVERSITY                2868
NORTHWELL HEALTH                      2573
HOMEMAKER                             2312
PAGNY                                 2105
PRP RESTAURANT INC                    1698
HAMBURGER WOOLEN CO, INC              1489
SUFFOLK COUNTY COMMUNITY COLLEGE      1383
USPS                                  1204
FEDEX                                 1184
NEW YORK LIFE INSURANCE COMPANY       1166
SYOSSET T A                           1088
Name: count, dtype: int64

In [13]:
bellmore_df = df[df.column_11=="11710"].copy()

In [15]:
committees_df = pd.read_csv("../references/committee_summary_2020.csv")
committees_df

,Link_Image,CMTE_ID,CMTE_NM,CMTE_TP,CMTE_DSGN,CMTE_FILING_FREQ,CMTE_ST1,CMTE_ST2,CMTE_CITY,CMTE_ST,...,ITEM_CONVN_EXP_DISB,ITEM_OTHER_DISB,SUBTTL_CONVN_EXP_DISB,TTL_EXP_SUBJECT_LIMITS,UNITEM_CONVN_EXP_DISB,UNITEM_OTHER_DISB,TTL_COMMUNICATION_COST,COH_BOY,COH_COY,ORG_TP
0,https://www.fec.gov/data/committee/C00729137/?...,C00729137,FRIENDS OF ERIC MOYER,H,P,Q,1951 WOODBRIAR DR,NaN,KETTERING,OH,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://www.fec.gov/data/committee/C00659813/?...,C00659813,CATHERINE KRANTZ FOR CONGRESS,H,P,T,314A WEST LENNON DR. #191,NaN,EMORY,TX,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://www.fec.gov/data/committee/C00647180/?...,C00647180,"FLORIDA FIRST, FLORIDA FOREVER",O,U,T,610 S. BOULEVARD,NaN,TAMPA,FL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://www.fec.gov/data/committee/C00711648/?...,C00711648,BLACK TECH MATTERS,N,U,Q,435 21ST STREET NE,NaN,WASHINGTON,DC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://www.fec.gov/data/committee/C00611962/?...,C00611962,TURNER VICTORY FUND,N,J,Q,824 S MILLEDGE AVE STE 101,NaN,ATHENS,GA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13541,https://www.fec.gov/data/committee/C00699306/?...,C00699306,DEMOCROC,N,U,Q,26895 ALISO CREEK RD SUITE B-282,NaN,ALISO VIEJO,CA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13542,https://www.fec.gov/data/committee/C00699314/?...,C00699314,ASIAN AMERICANS RISING,V,U,Q,220 NEWPORT CENTER DR #11-173,NaN,NEWPORT BEACH,CA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13543,https://www.fec.gov/data/committee/C00699819/?...,C00699819,POLICE OFFICERS SUPPORT FUND PAC,O,U,T,"7901 4TH ST. N, STE 4072",NaN,ST. PETERSBURG,FL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13544,https://www.fec.gov/data/committee/C00701797/?...,C00701797,EQUALITY CALIFORNIA VOTES,O,U,M,"515 S. FIGUEROA ST., STE. 1110",NaN,LOS ANGELES,CA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
committees_df

,Link_Image,CMTE_ID,CMTE_NM,CMTE_TP,CMTE_DSGN,CMTE_FILING_FREQ,CMTE_ST1,CMTE_ST2,CMTE_CITY,CMTE_ST,...,ITEM_CONVN_EXP_DISB,ITEM_OTHER_DISB,SUBTTL_CONVN_EXP_DISB,TTL_EXP_SUBJECT_LIMITS,UNITEM_CONVN_EXP_DISB,UNITEM_OTHER_DISB,TTL_COMMUNICATION_COST,COH_BOY,COH_COY,ORG_TP
0,https://www.fec.gov/data/committee/C00729137/?...,C00729137,FRIENDS OF ERIC MOYER,H,P,Q,1951 WOODBRIAR DR,NaN,KETTERING,OH,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://www.fec.gov/data/committee/C00659813/?...,C00659813,CATHERINE KRANTZ FOR CONGRESS,H,P,T,314A WEST LENNON DR. #191,NaN,EMORY,TX,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://www.fec.gov/data/committee/C00647180/?...,C00647180,"FLORIDA FIRST, FLORIDA FOREVER",O,U,T,610 S. BOULEVARD,NaN,TAMPA,FL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://www.fec.gov/data/committee/C00711648/?...,C00711648,BLACK TECH MATTERS,N,U,Q,435 21ST STREET NE,NaN,WASHINGTON,DC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://www.fec.gov/data/committee/C00611962/?...,C00611962,TURNER VICTORY FUND,N,J,Q,824 S MILLEDGE AVE STE 101,NaN,ATHENS,GA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13541,https://www.fec.gov/data/committee/C00699306/?...,C00699306,DEMOCROC,N,U,Q,26895 ALISO CREEK RD SUITE B-282,NaN,ALISO VIEJO,CA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13542,https://www.fec.gov/data/committee/C00699314/?...,C00699314,ASIAN AMERICANS RISING,V,U,Q,220 NEWPORT CENTER DR #11-173,NaN,NEWPORT BEACH,CA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13543,https://www.fec.gov/data/committee/C00699819/?...,C00699819,POLICE OFFICERS SUPPORT FUND PAC,O,U,T,"7901 4TH ST. N, STE 4072",NaN,ST. PETERSBURG,FL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13544,https://www.fec.gov/data/committee/C00701797/?...,C00701797,EQUALITY CALIFORNIA VOTES,O,U,M,"515 S. FIGUEROA ST., STE. 1110",NaN,LOS ANGELES,CA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
bellmore_df.merge(committees_df, left_on=["column_1"], right_on=["CMTE_ID"])[["CMTE_NM"]].value_counts()

CMTE_NM                                                                                          
ACTBLUE                                                                                              1796
TRUMP MAKE AMERICA GREAT AGAIN COMMITTEE                                                              160
DONALD J. TRUMP FOR PRESIDENT, INC.                                                                   112
PERDUE FOR SENATE                                                                                      89
NATIONAL ASSOCIATION OF LETTER CARRIERS OF U.S.A. POLITICAL FUND (LETTER CARRIER POLITICAL FUND)       64
MASON TENDERS DISTRICT COUNCIL OF GREATER NEW YORK & LI PAC                                            50
WINRED                                                                                                 29
EMILY'S LIST                                                                                           21
RETAIL, WHOLESALE & DEPARTMENT STORE UNION COMMITTEE O

In [62]:
df.groupby("EMPLOYER").size().sort_values().tail(50)

EMPLOYER
AMERICAN ARBITRATION ASSOCIATION       607
IRRELEVANT                             610
WIDGEON MGT CORP                       632
LVCSD                                  633
NYPD                                   642
ENDEAVOR AIR                           647
VERIZON                                647
GEICO                                  649
SUFFOLK COUNTY HEALTH SERVICES         656
CITY OF NEW YORK                       670
STATE FARM                             676
PROGRESSIVE                            680
SUNY STONY BROOK                       694
NASSAU COUNTY                          695
F A OF SUFFOLK CC                      708
SUFFOLK COUNTY                         733
CUNY                                   775
NEC CORPORATION OF AMERICA             781
NYC DOE                                795
HOFSTRA UNIVERSITY                     799
CMD25 APPAREL INC                      823
NOT-EMPLOYED                           824
NORTHWELL                              850
TH

In [64]:
df.groupby("OCCUPATION").size().sort_values().tail(50)

OCCUPATION
CARPENTER                   1425
INSURANCE                   1455
MUSICIAN                    1509
DIRECTOR                    1538
EDUCATOR                    1543
CLERK                       1561
CONSTRUCTION                1569
UNEMPLOYED                  1652
DESIGNER                    1729
SOFTWARE ENGINEER           1730
DENTIST                     1850
ELECTRICIAN                 1858
MD                          1881
ARTIST                      1918
PSYCHOTHERAPIST             1968
NURSE                       1992
SELF EMPLOYED               2000
AUDITOR                     2045
BUSINESS EXECUTIVE          2160
CONTRACTOR                  2167
FINANCE                     2177
REGISTERED NURSE            2231
SOCIAL WORKER               2289
RN                          2564
SELF-EMPLOYED               2784
BUSINESS OWNER              2803
ACCOUNTANT                  2891
PSYCHOLOGIST                3182
HOMEMAKER                   3193
CEO                         3260

In [69]:
df[df.OCCUPATION.fillna("").str.startswith("SOFTWARE")]["EMPLOYER"].value_counts()[:40]

EMPLOYER
ROCKIT SOLUTIONS             291
ABI RESEARCH                 241
SELF EMPLOYED                167
NOT EMPLOYED                 154
BROADRIDGE                   120
VTS                          112
SELF-EMPLOYED                101
BLOOMBERG LP                  90
EPIQ                          84
GOOGLE                        81
CLEVER DEVICES                60
INFORMATION REQUESTED         55
BLOOMBERG                     39
JP MORGAN CHASE               36
ANCHOR COMPUTER               35
NORTHWELL HEALTH              32
CENTERVIEW PARTNERS LLC       32
CONTINUITY LOGIC              31
SELF                          30
OUTBRAIN INC                  30
ETSY                          30
MORGAN STANLEY                29
MICROSOFT                     26
RATIONAL PATH INC.            26
SPACEX                        26
CODICS CORP                   25
PRISM VISUAL SOFTWARE INC     20
DRAFT                         20
VERINT SYSTEMS INC            20
RATIONAL PATH, INC.           20
F